In [1]:
import pandas as pd

In [4]:
energy_data = pd.read_csv(r"D:\A_Kashyap\UIC\Sem-3\Big Data Visualization\Chicago_Energy_Benchmarking_20241014.csv")

In [7]:
energy_data.head()

,Data Year,ID,Property Name,Reporting Status,Address,ZIP Code,Chicago Energy Rating,Exempt From Chicago Energy Rating,Community Area,Primary Property Type,...,Site EUI (kBtu/sq ft),Source EUI (kBtu/sq ft),Weather Normalized Site EUI (kBtu/sq ft),Weather Normalized Source EUI (kBtu/sq ft),Total GHG Emissions (Metric Tons CO2e),GHG Intensity (kg CO2e/sq ft),Latitude,Longitude,Location,Row_ID
0,2020,252064,Mansueto Library,Submitted Data,1100 E 57th St,60637,2.0,False,HYDE PARK,Library,...,240.8,323.6,246.0,329.9,1160.9,18.1,41.792213,-87.599950,"(41.79221307, -87.59994981)",2020-252064
1,2020,232458,Harper Square Cooperative,Submitted Data,4800 - 4850 S Lake Park Ave,60615,1.0,False,KENWOOD,Multifamily Housing,...,95.5,146.0,100.3,150.7,4871.7,7.8,41.807475,-87.591264,"(41.80747487, -87.59126397)",2020-232458
2,2020,254616,Former Coyne College,Submitted Data,330 N Green St,60607,2.0,False,NEAR WEST SIDE,Office,...,55.5,148.3,56.7,151.8,4581.4,8.2,41.873335,-87.651021,"(41.873335, -87.65102071)",2020-254616
3,2020,103812,400 W Superior St,Submitted Data,400 W Superior St,60654,3.0,False,NEAR NORTH SIDE,Office,...,61.4,151.8,63.0,154.8,1092.1,8.4,41.895752,-87.638901,"(41.89575232, -87.638901)",2020-103812
4,2020,254073,Blue Moon Lofts,Submitted Data,215 N. Aberdeen St.,60607,4.0,False,NEAR WEST SIDE,Multifamily Housing,...,29.4,64.9,29.6,64.3,295.8,3.6,41.874295,-87.650175,"(41.87429514, -87.65017516)",2020-254073


In [8]:
# Check for missing values in the dataset
missing_data = energy_data.isnull().sum()

# Display columns with missing data to see what needs cleaning
missing_data[missing_data > 0]


Property Name                                  2670
ZIP Code                                       1321
Chicago Energy Rating                          8570
Exempt From Chicago Energy Rating              7278
Community Area                                  308
Primary Property Type                          4176
Gross Floor Area - Buildings (sq ft)           1371
Year Built                                     4178
# of Buildings                                 3600
Water Use (kGal)                              21488
ENERGY STAR Score                              7505
Electricity Use (kBtu)                         4462
Natural Gas Use (kBtu)                         5777
District Steam Use (kBtu)                     19182
District Chilled Water Use (kBtu)             19054
All Other Fuel Use (kBtu)                     24809
Site EUI (kBtu/sq ft)                          4604
Source EUI (kBtu/sq ft)                        5463
Weather Normalized Site EUI (kBtu/sq ft)       5714
Weather Norm

In [9]:
# Drop rows with missing latitude or longitude
cleaned_data = energy_data.dropna(subset=['Latitude', 'Longitude'])

# Drop columns with excessive missing values or unnecessary for visualization
columns_to_drop = [
    'Property Name', 'ZIP Code', 'Exempt From Chicago Energy Rating', 
    'Community Area', 'Gross Floor Area - Buildings (sq ft)', 'Year Built', 
    '# of Buildings', 'Water Use (kGal)', 'ENERGY STAR Score', 'Electricity Use (kBtu)', 
    'Natural Gas Use (kBtu)', 'District Steam Use (kBtu)', 'District Chilled Water Use (kBtu)', 
    'All Other Fuel Use (kBtu)', 'Weather Normalized Site EUI (kBtu/sq ft)', 
    'Weather Normalized Source EUI (kBtu/sq ft)', 'Location', 'Row_ID'
]

# Keep only essential columns for map visualization
cleaned_data = cleaned_data.drop(columns=columns_to_drop)

# Check the cleaned data structure
cleaned_data.head()


,Data Year,ID,Reporting Status,Address,Chicago Energy Rating,Primary Property Type,Site EUI (kBtu/sq ft),Source EUI (kBtu/sq ft),Total GHG Emissions (Metric Tons CO2e),GHG Intensity (kg CO2e/sq ft),Latitude,Longitude
0,2020,252064,Submitted Data,1100 E 57th St,2.0,Library,240.8,323.6,1160.9,18.1,41.792213,-87.599950
1,2020,232458,Submitted Data,4800 - 4850 S Lake Park Ave,1.0,Multifamily Housing,95.5,146.0,4871.7,7.8,41.807475,-87.591264
2,2020,254616,Submitted Data,330 N Green St,2.0,Office,55.5,148.3,4581.4,8.2,41.873335,-87.651021
3,2020,103812,Submitted Data,400 W Superior St,3.0,Office,61.4,151.8,1092.1,8.4,41.895752,-87.638901
4,2020,254073,Submitted Data,215 N. Aberdeen St.,4.0,Multifamily Housing,29.4,64.9,295.8,3.6,41.874295,-87.650175


In [13]:
import folium

# Create a base map centered around Chicago
chicago_map = folium.Map(location=[41.8781, -87.6298], zoom_start=11)

# Function to color markers based on Site EUI values
def color_marker(eui):
    if eui < 50:
        return 'green'
    elif eui < 100:
        return 'orange'
    else:
        return 'red'

# Add markers to the map
for index, row in cleaned_data.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        popup=f"Address: {row['Address']}<br>Site EUI: {row['Site EUI (kBtu/sq ft)']}<br>GHG Emissions: {row['Total GHG Emissions (Metric Tons CO2e)']}<br>Energy Rating: {row['Chicago Energy Rating']}",
        color=color_marker(row['Site EUI (kBtu/sq ft)']),
        fill=True,
        fill_opacity=0.7
    ).add_to(chicago_map)

# Save the map to an HTML file and display
map_file_path = r'D:\A_Kashyap\UIC\Sem-3\Big Data Visualization\chicago_energy_map.html'
chicago_map.save(map_file_path)

map_file_path  # Returning the path to the map file for download or preview


'D:\\A_Kashyap\\UIC\\Sem-3\\Big Data Visualization\\chicago_energy_map.html'

In [12]:
!pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/ae/6d/18a7546e1748ecdd6ed7cd00d3f183faf1df08bd4f5e5e0eb3e72458b862/folium-0.17.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/fc/be/720f85abacd654ec86f1431bc7c004eae74417bd9d0e7a2bc43601062627/branca-0.8.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/108.4 kB ? eta -:--:--
   ------- ------------------------------- 20.5/108.4 kB 682.7 kB/s eta 0:00:01
   ------------------------- ------------- 71.7/108.4 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------- 108.4/108.4 kB 1.1 MB/s eta 0:00:00
